In [46]:
from tqdm.notebook import tqdm
import re
import threading
import time
import numpy as np
import pandas as pd
import redis
from ast import literal_eval
from concurrent.futures import ThreadPoolExecutor, as_completed

In [49]:
rd = redis.Redis(host = "107.22.67.15",port = 6379,password = "cryogen",charset="utf-8", decode_responses=True)
rd.ping()

True

In [38]:
df = pd.read_csv("df_ExKey.csv",converters={'auth_keywords_list': literal_eval,"extract_keywords" : literal_eval})
df

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,scopus_id,author_name,title,abstracts,related_field,auth_keywords,publish_day,publish_month,publish_year,ref_count,ref_authors,ref_titles,cited_count,auth_keywords_list,extract_keywords,correlation_keywords
0,0,0,0,85077976956,Pongpirul K.;Lungren M.P.,Public health and international epidemiology f...,NaN,MEDI,NaN,31,12,2018,76,Dreyer K.;De Wijkerslooth T.R.;Olszewski J.;Al...,The untilled fields of public health;Committee...,1,[],"[international, epidemiology, radiology, Publi...",0.000000
1,1,1,1,85060936020,Pratumsiri T.;Janpugdee P.,Flexible Printed Active Antenna for Digital Te...,This paper presents the development of a flexi...,ENGI;MATE,NaN,31,12,2018,4,Pratumsiri T.;Janpugdee P.,Development of built-in low-profile antenna fo...,1,[],"[directional radiation pattern, film radiating...",0.000000
2,2,2,2,85052201238,Phuakpunk K.;Chalermsinsuwan B.;Putivisutisak ...,Parametric study of hydrogen production via so...,Computational fluid dynamics was applied for s...,CHEM;CENG;ENGI,Circulating fluidized bed;Computational fluid ...,31,12,2018,42,Cunha A.F.;Samruamphianskun T.;Alonso M.;Harri...,Capture of CO2from combustion gases in a fluid...,21,"[Circulating fluidized bed, Computational flui...","[hydrogen production, computational fluid dyna...",0.525092
3,3,3,3,85051498032,Saengkaew J.;Le D.;Samart C.;Sawada H.;Nishida...,Superhydrophobic coating from fluoroalkylsilan...,A superhydrophobic/superoleophilic mesh was su...,CHEM;PHYS;PHYS;PHYS;MATE,Encapsulation;Fluoroalkylsilane;Natural rubber...,31,12,2018,45,Saito T.;Schondelmaier D.;Mabry J.M.;Lin T.;Wa...,Ceramic membrane performance in microfiltratio...,37,"[Encapsulation, Fluoroalkylsilane, Natural rub...","[exhibited superhydrophobic superoleophilic, c...",0.592623
4,4,4,4,85050678366,Teengam P.;Siangproh W.;Tuantranont A.;Vilaiva...,Electrochemical impedance-based DNA sensor usi...,A label-free electrochemical DNA sensor based ...,CHEM;BIOC;ENVI;CHEM,acpcPNA;Electrochemical impedance spectroscopy...,31,12,2018,55,Nuanyai T.;Boey F.;Lawrence M.F.;Svenson S.B.;...,The diagnosis and misdiagnosis of tuberculosis...,68,"[acpcPNA, Electrochemical impedance spectrosco...","[acids tuberculosis, based electrochemical dna...",0.479123
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39411,39411,784,9595,85190071715,Volk T.M.,"Music, Education, and Multiculturalism: Founda...","In today’s multi-ethnic classroom, multicultur...",ARTS,NaN,1,1,2023,729,Gollnick D.M.;Brooks-Baham E.S.;Rothstein E.;B...,Nationalizing Education;Changing Conceptions o...,107,[],"[section outlines multiculturalism, especially...",0.000000
39412,39412,785,9596,85190074221,Fischer O.,Morphosyntactic Change: Functional and Formal ...,This book presents a critical comparison of th...,ARTS;SOCI,NaN,1,1,2023,512,Knight C.;Marchand H.;Rosch E.;Detges U.;Choms...,A lovely little example: Word order options an...,180,[],"[semantics pragmatics, meaning linguistic, for...",0.000000
39413,39413,786,9597,85190076689,Hudson R.,Language Networks: The New Word Grammar,This book argues that language is a network of...,ARTS;SOCI,NaN,1,1,2023,302,Jorgensen E.;O'Connor M.;Luger G.;Logie R.;Col...,Valency and valency grammar;The representation...,207,[],"[held view language, knowledge language puts, ...",0.000000
39414,39414,787,9598,85190200500,Button K.,Transport economics: 4th edition,Transport Economics is a revised and refined f...,ECON;BUSI;SOCI,NaN,20,5,2022,547,Consultancy J.;Demsetz H.;Linneker B.J.;Evans ...,Some economic problems of the transport indust...,315,[],"[ensure social environmental, economics revise...",0.000000


In [39]:
df = df.drop(["Unnamed: 0.2","Unnamed: 0.1","Unnamed: 0"],axis = 1).set_index("scopus_id")
df

,author_name,title,abstracts,related_field,auth_keywords,publish_day,publish_month,publish_year,ref_count,ref_authors,ref_titles,cited_count,auth_keywords_list,extract_keywords,correlation_keywords
scopus_id,,,,,,,,,,,,,,,
85077976956,Pongpirul K.;Lungren M.P.,Public health and international epidemiology f...,NaN,MEDI,NaN,31,12,2018,76,Dreyer K.;De Wijkerslooth T.R.;Olszewski J.;Al...,The untilled fields of public health;Committee...,1,[],"[international, epidemiology, radiology, Publi...",0.000000
85060936020,Pratumsiri T.;Janpugdee P.,Flexible Printed Active Antenna for Digital Te...,This paper presents the development of a flexi...,ENGI;MATE,NaN,31,12,2018,4,Pratumsiri T.;Janpugdee P.,Development of built-in low-profile antenna fo...,1,[],"[directional radiation pattern, film radiating...",0.000000
85052201238,Phuakpunk K.;Chalermsinsuwan B.;Putivisutisak ...,Parametric study of hydrogen production via so...,Computational fluid dynamics was applied for s...,CHEM;CENG;ENGI,Circulating fluidized bed;Computational fluid ...,31,12,2018,42,Cunha A.F.;Samruamphianskun T.;Alonso M.;Harri...,Capture of CO2from combustion gases in a fluid...,21,"[Circulating fluidized bed, Computational flui...","[hydrogen production, computational fluid dyna...",0.525092
85051498032,Saengkaew J.;Le D.;Samart C.;Sawada H.;Nishida...,Superhydrophobic coating from fluoroalkylsilan...,A superhydrophobic/superoleophilic mesh was su...,CHEM;PHYS;PHYS;PHYS;MATE,Encapsulation;Fluoroalkylsilane;Natural rubber...,31,12,2018,45,Saito T.;Schondelmaier D.;Mabry J.M.;Lin T.;Wa...,Ceramic membrane performance in microfiltratio...,37,"[Encapsulation, Fluoroalkylsilane, Natural rub...","[exhibited superhydrophobic superoleophilic, c...",0.592623
85050678366,Teengam P.;Siangproh W.;Tuantranont A.;Vilaiva...,Electrochemical impedance-based DNA sensor usi...,A label-free electrochemical DNA sensor based ...,CHEM;BIOC;ENVI;CHEM,acpcPNA;Electrochemical impedance spectroscopy...,31,12,2018,55,Nuanyai T.;Boey F.;Lawrence M.F.;Svenson S.B.;...,The diagnosis and misdiagnosis of tuberculosis...,68,"[acpcPNA, Electrochemical impedance spectrosco...","[acids tuberculosis, based electrochemical dna...",0.479123
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85190071715,Volk T.M.,"Music, Education, and Multiculturalism: Founda...","In today’s multi-ethnic classroom, multicultur...",ARTS,NaN,1,1,2023,729,Gollnick D.M.;Brooks-Baham E.S.;Rothstein E.;B...,Nationalizing Education;Changing Conceptions o...,107,[],"[section outlines multiculturalism, especially...",0.000000
85190074221,Fischer O.,Morphosyntactic Change: Functional and Formal ...,This book presents a critical comparison of th...,ARTS;SOCI,NaN,1,1,2023,512,Knight C.;Marchand H.;Rosch E.;Detges U.;Choms...,A lovely little example: Word order options an...,180,[],"[semantics pragmatics, meaning linguistic, for...",0.000000
85190076689,Hudson R.,Language Networks: The New Word Grammar,This book argues that language is a network of...,ARTS;SOCI,NaN,1,1,2023,302,Jorgensen E.;O'Connor M.;Luger G.;Logie R.;Col...,Valency and valency grammar;The representation...,207,[],"[held view language, knowledge language puts, ...",0.000000


In [40]:
df = df.fillna("")
df

,author_name,title,abstracts,related_field,auth_keywords,publish_day,publish_month,publish_year,ref_count,ref_authors,ref_titles,cited_count,auth_keywords_list,extract_keywords,correlation_keywords
scopus_id,,,,,,,,,,,,,,,
85077976956,Pongpirul K.;Lungren M.P.,Public health and international epidemiology f...,,MEDI,,31,12,2018,76,Dreyer K.;De Wijkerslooth T.R.;Olszewski J.;Al...,The untilled fields of public health;Committee...,1,[],"[international, epidemiology, radiology, Publi...",0.000000
85060936020,Pratumsiri T.;Janpugdee P.,Flexible Printed Active Antenna for Digital Te...,This paper presents the development of a flexi...,ENGI;MATE,,31,12,2018,4,Pratumsiri T.;Janpugdee P.,Development of built-in low-profile antenna fo...,1,[],"[directional radiation pattern, film radiating...",0.000000
85052201238,Phuakpunk K.;Chalermsinsuwan B.;Putivisutisak ...,Parametric study of hydrogen production via so...,Computational fluid dynamics was applied for s...,CHEM;CENG;ENGI,Circulating fluidized bed;Computational fluid ...,31,12,2018,42,Cunha A.F.;Samruamphianskun T.;Alonso M.;Harri...,Capture of CO2from combustion gases in a fluid...,21,"[Circulating fluidized bed, Computational flui...","[hydrogen production, computational fluid dyna...",0.525092
85051498032,Saengkaew J.;Le D.;Samart C.;Sawada H.;Nishida...,Superhydrophobic coating from fluoroalkylsilan...,A superhydrophobic/superoleophilic mesh was su...,CHEM;PHYS;PHYS;PHYS;MATE,Encapsulation;Fluoroalkylsilane;Natural rubber...,31,12,2018,45,Saito T.;Schondelmaier D.;Mabry J.M.;Lin T.;Wa...,Ceramic membrane performance in microfiltratio...,37,"[Encapsulation, Fluoroalkylsilane, Natural rub...","[exhibited superhydrophobic superoleophilic, c...",0.592623
85050678366,Teengam P.;Siangproh W.;Tuantranont A.;Vilaiva...,Electrochemical impedance-based DNA sensor usi...,A label-free electrochemical DNA sensor based ...,CHEM;BIOC;ENVI;CHEM,acpcPNA;Electrochemical impedance spectroscopy...,31,12,2018,55,Nuanyai T.;Boey F.;Lawrence M.F.;Svenson S.B.;...,The diagnosis and misdiagnosis of tuberculosis...,68,"[acpcPNA, Electrochemical impedance spectrosco...","[acids tuberculosis, based electrochemical dna...",0.479123
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85190071715,Volk T.M.,"Music, Education, and Multiculturalism: Founda...","In today’s multi-ethnic classroom, multicultur...",ARTS,,1,1,2023,729,Gollnick D.M.;Brooks-Baham E.S.;Rothstein E.;B...,Nationalizing Education;Changing Conceptions o...,107,[],"[section outlines multiculturalism, especially...",0.000000
85190074221,Fischer O.,Morphosyntactic Change: Functional and Formal ...,This book presents a critical comparison of th...,ARTS;SOCI,,1,1,2023,512,Knight C.;Marchand H.;Rosch E.;Detges U.;Choms...,A lovely little example: Word order options an...,180,[],"[semantics pragmatics, meaning linguistic, for...",0.000000
85190076689,Hudson R.,Language Networks: The New Word Grammar,This book argues that language is a network of...,ARTS;SOCI,,1,1,2023,302,Jorgensen E.;O'Connor M.;Luger G.;Logie R.;Col...,Valency and valency grammar;The representation...,207,[],"[held view language, knowledge language puts, ...",0.000000


In [41]:
df["auth_keywords_list"] = df["auth_keywords_list"].str.join(";")
df["extract_keywords"] = df["extract_keywords"].str.join(";")
df

,author_name,title,abstracts,related_field,auth_keywords,publish_day,publish_month,publish_year,ref_count,ref_authors,ref_titles,cited_count,auth_keywords_list,extract_keywords,correlation_keywords
scopus_id,,,,,,,,,,,,,,,
85077976956,Pongpirul K.;Lungren M.P.,Public health and international epidemiology f...,,MEDI,,31,12,2018,76,Dreyer K.;De Wijkerslooth T.R.;Olszewski J.;Al...,The untilled fields of public health;Committee...,1,,international;epidemiology;radiology;Public;he...,0.000000
85060936020,Pratumsiri T.;Janpugdee P.,Flexible Printed Active Antenna for Digital Te...,This paper presents the development of a flexi...,ENGI;MATE,,31,12,2018,4,Pratumsiri T.;Janpugdee P.,Development of built-in low-profile antenna fo...,1,,directional radiation pattern;film radiating e...,0.000000
85052201238,Phuakpunk K.;Chalermsinsuwan B.;Putivisutisak ...,Parametric study of hydrogen production via so...,Computational fluid dynamics was applied for s...,CHEM;CENG;ENGI,Circulating fluidized bed;Computational fluid ...,31,12,2018,42,Cunha A.F.;Samruamphianskun T.;Alonso M.;Harri...,Capture of CO2from combustion gases in a fluid...,21,Circulating fluidized bed;Computational fluid ...,hydrogen production;computational fluid dynami...,0.525092
85051498032,Saengkaew J.;Le D.;Samart C.;Sawada H.;Nishida...,Superhydrophobic coating from fluoroalkylsilan...,A superhydrophobic/superoleophilic mesh was su...,CHEM;PHYS;PHYS;PHYS;MATE,Encapsulation;Fluoroalkylsilane;Natural rubber...,31,12,2018,45,Saito T.;Schondelmaier D.;Mabry J.M.;Lin T.;Wa...,Ceramic membrane performance in microfiltratio...,37,Encapsulation;Fluoroalkylsilane;Natural rubber...,exhibited superhydrophobic superoleophilic;com...,0.592623
85050678366,Teengam P.;Siangproh W.;Tuantranont A.;Vilaiva...,Electrochemical impedance-based DNA sensor usi...,A label-free electrochemical DNA sensor based ...,CHEM;BIOC;ENVI;CHEM,acpcPNA;Electrochemical impedance spectroscopy...,31,12,2018,55,Nuanyai T.;Boey F.;Lawrence M.F.;Svenson S.B.;...,The diagnosis and misdiagnosis of tuberculosis...,68,acpcPNA;Electrochemical impedance spectroscopy...,acids tuberculosis;based electrochemical dna;a...,0.479123
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85190071715,Volk T.M.,"Music, Education, and Multiculturalism: Founda...","In today’s multi-ethnic classroom, multicultur...",ARTS,,1,1,2023,729,Gollnick D.M.;Brooks-Baham E.S.;Rothstein E.;B...,Nationalizing Education;Changing Conceptions o...,107,,section outlines multiculturalism;especially m...,0.000000
85190074221,Fischer O.,Morphosyntactic Change: Functional and Formal ...,This book presents a critical comparison of th...,ARTS;SOCI,,1,1,2023,512,Knight C.;Marchand H.;Rosch E.;Detges U.;Choms...,A lovely little example: Word order options an...,180,,semantics pragmatics;meaning linguistic;formal...,0.000000
85190076689,Hudson R.,Language Networks: The New Word Grammar,This book argues that language is a network of...,ARTS;SOCI,,1,1,2023,302,Jorgensen E.;O'Connor M.;Luger G.;Logie R.;Col...,Valency and valency grammar;The representation...,207,,held view language;knowledge language puts;lan...,0.000000


In [63]:
def send(idx,mapping):
    rd.hset( f"visualize:{idx}",mapping = mapping)
    
with tqdm(total = len(df)) as pbar:
    with ThreadPoolExecutor(max_workers=256) as executor:
        future = [executor.submit(send,idx,row.to_dict()) for idx,row in df.iterrows()]
        for f in as_completed(future):
            pbar.update(1)

  0%|          | 0/39416 [00:00<?, ?it/s]